In [ ]:
!pip install lime
!pip install datacleaner
!pip install scikit-learn==1.3.1

In [ ]:
import pandas as pd
import numpy as np
from datacleaner import autoclean
import statistics
from collections import Counter
import math
from sklearn.feature_selection import mutual_info_classif
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import pickle
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
data=pd.read_csv('/content/train.csv')


    ID - ID of the patient
    A1_Score to A10_Score - Score based on Autism Spectrum Quotient (AQ) 10 item screening tool
    age - Age of the patient in years
    gender - Gender of the patient
    ethnicity - Ethnicity of the patient
    jaundice - Whether the patient had jaundice at the time of birth
    autism - Whether an immediate family member has been diagnosed with autism
    contry_of_res - Country of residence of the patient
    used_app_before - Whether the patient has undergone a screening test before
    result - Score for AQ1-10 screening test
    age_desc - Age of the patient
    relation - Relation of patient who completed the test
    Class/ASD - Classified result as 0 or 1. Here 0 represents No and 1 represents Yes. This is the target column, and during submission submit the values as 0 or 1 only.



In [ ]:
data=data.drop('ID',axis=1)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data['ethnicity'].unique()

In [ ]:
c=data.columns
c

In [ ]:
data=data[data['age']>=18]
data=data[data['result']>=0]
data

In [ ]:
for i in range(0,len(c)-1):
  fig = px.box(data, y=c[i])
  fig.show()

In [ ]:
for i in c:
  if '?' in list(data[i].unique()):
    m=(statistics.mode(list(data[i])))
    data=data.replace({i:'?'},m)

In [ ]:
data

In [ ]:
data.to_csv('Dav_data.csv')

In [ ]:
data=autoclean(data)

In [ ]:
data

In [ ]:
dict={}
for i in data:
  dict[i]=(Counter(list(data[i])))
  print((Counter(list(data[i]))))

In [ ]:
dict_yes={}
for i in data:
  dict_yes[i]=(Counter(list(data[data['Class/ASD']==1][i])))
  print((Counter(list(data[data['Class/ASD']==1][i]))))

In [ ]:
dict_no={}
for i in data:
  dict_no[i]=(Counter(list(data[data['Class/ASD']==0][i])))
  print((Counter(list(data[data['Class/ASD']==0][i]))))

In [ ]:
yes_count=dict_yes['Class/ASD'][1]
yes_count

In [ ]:
no_count=dict_no['Class/ASD'][0]
no_count

In [ ]:
def compute(x,y):
  if x==0 or y==0:
    return 1
  t=x+y
  x=x/t
  y=y/t
  return -(x*math.log2(x)+y*math.log2(y))

In [ ]:
total_entropy=compute(yes_count,no_count)

In [ ]:
total_entropy

A1 Score

In [ ]:
entropy={}

In [ ]:
for i in c:
  l=[]
  for j in dict_yes[i]:
    l.append(compute(dict_yes[i][j],dict_no[i][j]))
  e=0
  m=0
  for k in dict_yes[i]:
    e=e+dict[i][k]*l[m]/len(data)
    m+=1
  entropy[i]=total_entropy-e


In [ ]:
entropy

In [ ]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
target_column = 'Class/ASD'
X = data.drop(target_column,axis=1)
y = data[target_column]
d_c=[True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
d_c[10]=False
d_c[17]=False
mutual_info_scores = mutual_info_classif(X, y, discrete_features=d_c)
information={}
for i in range(0,len(c)-1):
  information[c[i]]=round(mutual_info_scores[i],4)
information


In [ ]:
for i in c[0:len(c)-1]:
  if information[i]<=0.01:
    data=data.drop(i,axis=1)

In [ ]:
data.info()

In [ ]:
data.to_csv('Final_Data.csv')

AFTER PREPROCESSING

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
x=data.drop('Class/ASD',axis=1)
y=data['Class/ASD']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.to_csv('x_train.csv')

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
y_train.to_csv('y_train.csv')

Logistic Regression

In [ ]:
lr_model = LogisticRegression()
lr_model.fit(x_train, y_train)
lr_predictions = lr_model.predict(x_test)
lr_accuracy = accuracy_score(y_test, lr_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy:.2f}")


Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create and train the model
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)

# Make predictions
dt_predictions = dt_model.predict(x_test)

# Evaluate the model
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f"Decision Tree Accuracy: {dt_accuracy:.2f}")


Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create and train the model
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(x_test)

# Evaluate the model
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")


 Gradient Boosting (XGBoost):

In [ ]:
import xgboost as xgb

# Create and train the model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)

# Make predictions
xgb_predictions = xgb_model.predict(x_test)

# Evaluate the model
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
print(f"XGBoost Accuracy: {xgb_accuracy:.2f}")


Support Vector Machines (SVM)

In [ ]:
from sklearn.svm import SVC

# Create and train the model
svm_model = SVC(probability=True)
svm_model.fit(x_train, y_train)

# Make predictions
svm_predictions = svm_model.predict(x_test)

# Evaluate the model
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f"SVM Accuracy: {svm_accuracy:.2f}")


k-Nearest Neighbors (KNN):

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create and train the model
knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

# Make predictions
knn_predictions = knn_model.predict(x_test)

# Evaluate the model
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f"K-Nearest Neighbors Accuracy: {knn_accuracy:.2f}")


Naive bayers

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

# Make predictions
nb_predictions = nb_model.predict(x_test)

# Evaluate the model
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f"Naive Bayes Accuracy: {nb_accuracy:.2f}")

In [ ]:
accuraccy={}
accuraccy['Logistic Regression']=lr_accuracy
accuraccy['Decision tree']=dt_accuracy
accuraccy['Random forest']=rf_accuracy
accuraccy['SVM']=svm_accuracy
accuraccy['KNN']=knn_accuracy
accuraccy['Naive bayers']=nb_accuracy
accuraccy['XGBoost']=xgb_accuracy

In [ ]:
models={}
models['Logistic Regression']=lr_model
models['Decision tree']=dt_model
models['Random forest']=rf_model
models['SVM']=svm_model
models['KNN']=knn_model
models['Naive bayers']=nb_model
models['XGBoost']=xgb_model
models['acc']=accuraccy

In [ ]:
models

In [ ]:
dbfile = open('Data_Models.pkl', 'wb')

In [ ]:
pickle.dump(models, dbfile)
dbfile.close()

In [ ]:
k=list(x_test.to_numpy()[0])
k

In [ ]:
"""dbfile = open('Data_Models.pkl', 'rb')
db = pickle.load(dbfile)
for keys in db:
     print(db[keys].predict(x_test))
dbfile.close()"""

In [ ]:
x_train.values

In [ ]:
for model in db:
  try:
    explainer = LimeTabularExplainer(x_train.values, mode="classification", training_labels=y_train,
                                    feature_names=x_train.columns, random_state=42)
    instance_to_explain = x_test.iloc[0]
    def predict_fn(x):
        return db[model].predict_proba(x)
    explanation = explainer.explain_instance(instance_to_explain.values, predict_fn, num_features=len(x_train.columns))
    explanation.show_in_notebook()
  except:
    pass
